# Satellite Location Demo

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed, both in the past and in the future, in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics.

In [1]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import OrRd_9
from palettable.colorbrewer.diverging import BrBG_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [2]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

sentinel_2b = load.tle_file(sentinel_2b_url, reload=True, filename="sentinel_2b")

[#################################] 100% sentinel_2b


In [3]:
satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-06-12 04:08:31 UTC>

---

## Calculating the position of a satellite

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +- 1 day from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run).

In [4]:
# Calculate the current UTC time (without microseconds), then creating a time range +/- 24h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides)
eph = load('de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now - timedelta(days=1)
t2 = now + timedelta(days=1)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [5]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [6]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords, daytime

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [7]:
while timer <= t2:
    timer, longitude, latitude, coords, daytime = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=30) # ~7 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [8]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
0,SENTINEL-2B,2024-06-11 11:51:57+00:00,POINT (3.03133 69.39471),3.031333,69.394710,True,2024-06-11 11:51:57
1,SENTINEL-2B,2024-06-11 11:52:27+00:00,POINT (0.92155 67.76540),0.921554,67.765403,True,2024-06-11 11:52:27
2,SENTINEL-2B,2024-06-11 11:52:57+00:00,POINT (-0.92884 66.11479),-0.928840,66.114787,True,2024-06-11 11:52:57
3,SENTINEL-2B,2024-06-11 11:53:27+00:00,POINT (-2.56775 64.44689),-2.567752,64.446891,True,2024-06-11 11:53:27
4,SENTINEL-2B,2024-06-11 11:53:57+00:00,POINT (-4.03251 62.76479),-4.032507,62.764791,True,2024-06-11 11:53:57
...,...,...,...,...,...,...,...
5756,SENTINEL-2B,2024-06-13 11:49:57+00:00,POINT (167.77817 -41.94441),167.778172,-41.944412,False,2024-06-13 11:49:57
5757,SENTINEL-2B,2024-06-13 11:50:27+00:00,POINT (167.18726 -40.19322),167.187264,-40.193218,False,2024-06-13 11:50:27
5758,SENTINEL-2B,2024-06-13 11:50:57+00:00,POINT (166.61965 -38.43950),166.619646,-38.439502,False,2024-06-13 11:50:57
5759,SENTINEL-2B,2024-06-13 11:51:27+00:00,POINT (166.07287 -36.68346),166.072865,-36.683455,False,2024-06-13 11:51:27


Let's refine our area of interest to see when/where satellites pass overhead. We can use a large bounding box of South Asia for demonstration purposes.

In [9]:
# South Asia bbox

min_lon = 53.45
max_lon = 98.16
min_lat = 3.89
max_lat = 40.24

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

[53.45, 98.16, 3.89, 40.24]

In [10]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()
aoi

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
546,SENTINEL-2B,2024-06-11 16:24:57+00:00,POINT (90.57917 4.71387),90.579173,4.713868,False,2024-06-11 16:24:57
547,SENTINEL-2B,2024-06-11 16:25:27+00:00,POINT (90.18499 6.49231),90.184995,6.492309,False,2024-06-11 16:25:27
548,SENTINEL-2B,2024-06-11 16:25:57+00:00,POINT (89.78891 8.27058),89.788911,8.270581,False,2024-06-11 16:25:57
549,SENTINEL-2B,2024-06-11 16:26:27+00:00,POINT (89.39038 10.04860),89.390381,10.048602,False,2024-06-11 16:26:27
550,SENTINEL-2B,2024-06-11 16:26:57+00:00,POINT (88.98884 11.82629),88.988841,11.826289,False,2024-06-11 16:26:57
...,...,...,...,...,...,...,...
5068,SENTINEL-2B,2024-06-13 06:05:57+00:00,POINT (67.84104 11.86058),67.841042,11.860578,True,2024-06-13 06:05:57
5069,SENTINEL-2B,2024-06-13 06:06:27+00:00,POINT (67.43943 10.08285),67.439430,10.082846,True,2024-06-13 06:06:27
5070,SENTINEL-2B,2024-06-13 06:06:57+00:00,POINT (67.04084 8.30478),67.040839,8.304777,True,2024-06-13 06:06:57
5071,SENTINEL-2B,2024-06-13 06:07:27+00:00,POINT (66.64471 6.52645),66.644705,6.526455,True,2024-06-13 06:07:27


Next, we can visualize these points on a map. They are color coded using a diverging color scale, with brown being before `now` and green being after `now`.

In [11]:
# This creates a range from 0-1 to define our colormap.
time_norm = (aoi.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, BrBG_9)

In [12]:
layer = ScatterplotLayer.from_geopandas(
    aoi,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame.

---

## Calculating the position of a satellite constellation

Usually, humanitarians aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [14]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

planet_swarm = load.tle_file(planet_url, reload=True, filename="planet_swarm")
planet_swarm[0]

[#################################] 100% planet_swarm


<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-06-12 02:27:34 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [15]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords, daytime = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=5) # ~ 1 min 45s

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [42]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [44]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
aoi_planet_day[aoi_planet_day["timestamp"] > now]

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
49708,SKYSAT-C16,2024-06-12 13:06:57+00:00,POINT (87.00313 38.87908),87.003135,38.879076,True,2024-06-12 13:06:57
51337,SKYSAT-C2,2024-06-12 13:56:57+00:00,POINT (95.07295 37.95610),95.072950,37.956098,True,2024-06-12 13:56:57
52824,SKYSAT-C16,2024-06-12 14:41:57+00:00,POINT (73.86640 30.48973),73.866397,30.489728,True,2024-06-12 14:41:57
52975,SKYSAT-C4,2024-06-12 14:46:57+00:00,POINT (84.88512 31.60898),84.885120,31.608985,True,2024-06-12 14:46:57
53083,FLOCK 4Y-33,2024-06-12 14:46:57+00:00,POINT (97.85033 36.25842),97.850326,36.258420,True,2024-06-12 14:46:57
...,...,...,...,...,...,...,...
91683,SKYSAT-C11,2024-06-13 10:26:57+00:00,POINT (58.70425 38.81331),58.704255,38.813307,True,2024-06-13 10:26:57
91685,SKYSAT-C9,2024-06-13 10:26:57+00:00,POINT (56.71764 38.04449),56.717644,38.044485,True,2024-06-13 10:26:57
91847,SKYSAT-C11,2024-06-13 10:31:57+00:00,POINT (54.16280 19.71644),54.162801,19.716444,True,2024-06-13 10:31:57
92012,SKYSAT-C10,2024-06-13 10:36:57+00:00,POINT (54.60599 32.14601),54.605989,32.146006,True,2024-06-13 10:36:57


In [45]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet = (aoi_planet_day.timestamp - t1) / (t2 - t1)

colors_planet = apply_continuous_cmap(time_norm_planet, BrBG_9)

Let's visualize this with the same color scheme we used before.

In [46]:
layer2 = ScatterplotLayer.from_geopandas(
    aoi_planet_day,
    # extensions=[filter_extension],
    get_fill_color=colors_planet,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m2 = Map(
    layer2,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

In reality, satellites are taking more images than the amount of time steps that we arbitrarily set. A line would better represent the path these satellites take. The area of capture of satellites also varies, which a point doesn't represent.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.

In [54]:
# geo_df2 = planet_swarm_gdf.groupby('satellite')['coordinates'].apply(lambda x: LineString(x.tolist()))
# geo_df2 = gpd.GeoDataFrame(geo_df2, geometry='coordinates')
# geo_df2 = geo_df2.reset_index()

In [55]:
# viz(geo_df2[geo_df2["satellite"] == "FLOCK 4Q-1"])